In [11]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
import timeit

In [12]:
def tri_solve_back(R, b):
    """Solves a system of linear equations via backwards substitution
    Args:
        R (np.array): upper triangular matrix
        b (np.array): input vector
    Returns:
        np.array: output vector
    """
    n = R.shape[1]
    x= np.zeros(n)
    for i in reversed(range(n)):
        x[i] = (b[i] - (np.dot(R[i,:],x))) / R[i, i]

In [13]:
def tri_solve_forw(L, b):
    """Solves a system of linear equations
    Args:
        L (np.array): lower triangular matrix
        b (np.array): input vector
    Returns:
        np.array: output vector
    """
    n = L.shape[0]
    x= np.zeros((n, 1))
    for i in range(n):
        x[i, 0] = (b[i] - (np.sum(L[i, :i] * x[:i, 0]))) / L[i, i]
    return x

#### Source: Alessandro Coretti

In [14]:
def lu(M_in, pv):
    """ 
    Compute the LU decomposition of a matrix M=(P)LU. 
    Input: 
    nxn matrix M_in; pv=T/F (pivoting).
    Output: 
    matrix M containing U in the upper triangle and 
    L below the main diagonal (NB: the diagonal elements 
    of L are equal to 1, not stored); 
    vector z describing the permutations: row i was 
    interchanged with row z[i].
    """
    M = np.copy(M_in)
    [m,n] = M.shape
    if m != n:
        raise SystemExit('M is not a square matrix: exit')
    if abs(linalg.det(M)) < 1e-8:
        raise SystemExit('M is singular')
    
    # initialize line swap vector
    z = np.arange(n)
    for j in range(n-1):
    # pivot search
        pivot = abs(M[j,j])
        p = j
        if pv:
            for i in range(j+1,n):
                tmp_pivot = abs(M[i,j])
                if tmp_pivot > pivot:
                    pivot = tmp_pivot
                    p = i # update index
        
        if pivot != 0:
            if pv:
            # line swapping (elegant)
                M[[j,p]] = M[[p,j]]
                z[j], z[p] = z[p], z[j]
            for i in range(j+1,n):
                M[i,j] = M[i,j] / M[j,j] # division by pivot, M[i,j] now stores L[i,j]
                M[i,j+1:n] = M[i,j+1:n] - M[i,j]*M[j,j+1:n]
        else:
            print(f'pivot = 0! j = {j}')
            
    return M, z


$$

M x = b \\
M = LU \\
Ly = b \\
Ux = y \\

$$

In [40]:
def LU_solve(M, b):
    M = lu(M, pv = True)[0]
    print(M)
    n = M.shape[0]
    L = np.eye(n) + np.tril(M, k = -1)
    
    # print(np.tril(M, k = -1))
    # print(L)
    U = np.triu(M)
    # print(U)
    y = tri_solve_forw(L, b)
    x = tri_solve_back(U, y)
    
    return x

In [41]:
M = np.array(([
        7, 3, -1, 2
    ],[
        3, 8, 1, -4
    ],[
        -1, 1, 4, -1
    ],[
        2, -4, -1, 6
    ]))
b = np.array([1, 2, 3, 4])

x  = LU_solve(M, b)
# print(x)


[[ 7  3 -1  2]
 [ 0  8  1 -4]
 [ 0  0  4 -1]
 [ 0  0  0  6]]
